# _utils.events

> TODO fill in description

In [ ]:
#| default_exp _utils.events

In [ ]:
#| hide
from nbdev.showdoc import *; 

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|export
from __future__ import annotations
import asyncio
from typing import List, Callable

import fbdev
from fbdev._utils.attr_container import AttrContainer

In [ ]:
#|hide
show_doc(fbdev._utils.events.await_multiple_events)

---

### await_multiple_events

>      await_multiple_events (*events)

In [ ]:
#|export
async def await_multiple_events(*events):
    try:
        event_await_tasks = []
        while not all([event.is_set() for event in events]): # In the off-chance that as asyncio.wait finishes, one of the events is cleared
            event_await_tasks.clear()
            for event in events:
                event_await_tasks.append(asyncio.create_task(event.wait()))
            await asyncio.wait(event_await_tasks)
    except asyncio.CancelledError:
        for task in event_await_tasks:
            task.cancel()
            try: await task
            except asyncio.CancelledError: pass
        raise

In [ ]:
#|hide
show_doc(fbdev._utils.events.await_any_event)

---

### await_any_event

>      await_any_event (*events)

In [ ]:
#|export
async def await_any_event(*events):
    try:
        event_await_tasks = [asyncio.create_task(event.wait()) for event in events]
        await asyncio.wait(event_await_tasks, return_when=asyncio.FIRST_COMPLETED)
    except asyncio.CancelledError:
        for task in event_await_tasks:
            task.cancel()
            try: await task
            except asyncio.CancelledError: pass
        raise

In [ ]:
ev1 = asyncio.Event()
async def foo1():
    await asyncio.sleep(0.5)
    ev1.set()
    
ev2 = asyncio.Event()
async def foo2():
    await asyncio.sleep(1)
    ev2.set()

asyncio.create_task(foo1())
asyncio.create_task(foo2())
await await_any_event(ev1, ev2)
print("Done waiting")
print("ev1 is set:", ev1.is_set())
print("ev2 is set:", ev2.is_set())

Done waiting
ev1 is set: True
ev2 is set: False


In [ ]:
#|hide
show_doc(fbdev._utils.events.ReadonlyEvent)

---

### ReadonlyEvent

>      ReadonlyEvent (event:asyncio.Event)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class ReadonlyEvent:
    def __init__(self, event: asyncio.Event):
        self._event = event

    def is_set(self):
        return self._event.is_set()

    async def wait(self):
        await self._event.wait()

In [ ]:
#|hide
show_doc(fbdev._utils.events.EventHandler)

---

### EventHandler

>      EventHandler (name)

*Subscribable events*

In [ ]:
#|export
class EventHandler:
    """Subscribable events"""
    def __init__(self, name):
        self._events: List[asyncio.Event] = []
        self._callbacks: List[Callable] = []
        self.name = name
    
    def subscribe(self):
        event = asyncio.Event()
        self._events.append(event)
        return event
    
    def register_callback(self, callback):
        self._callbacks.append(callback)

    def _trigger(self):
        for event in self._events:
            event.set()
        self._events.clear()
        for callback in self._callbacks:
            callback()
        
    def __str__(self):
        return f"EventHandler(name='{self.name}')"
    
    def __repr__(self):
        return str(self)

In [ ]:
#|hide
show_doc(fbdev._utils.events.EventCollection)

---

### EventCollection

>      EventCollection ()

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class EventCollection(AttrContainer):
    def __init__(self) -> None:
        super().__init__({}, obj_name="EventCollection")
    
    def _add_event(self, event_handler: EventHandler):
        self._set(event_handler.name, event_handler)